In [ ]:
# 環境準備
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 演習用のコンテンツを取得
!git clone https://github.com/yuki-mtmr/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70 (from 2)
Receiving objects: 100% (70/70), 86.43 KiB | 14.40 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import random
random.seed(0)

In [ ]:
# ベースラインモデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

# 質問①
2025年5月7日の東京の天気は大雨ですか?

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "2025年5月7日の東京の天気は大雨ですか?"
response = generate_output(question, system_prompt)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(response)

日本の天気予報は、現在の技術では2025年5月7日の天気を正確に予測することができません。天気予報は、複数の要因に基づいて行われますが、将来の天気は、多くの不確実性を含みます。

ただし、日本気象協会や気象庁などの公式ウェブサイトでは、将来の天気予報を提供しています。2025年5月7日の天気予報は、現在の情報では提供できませんが、将来の予報を確認するには、公式ウェブサイトを参照してください。


## 結果(ベースモデル)
llama3のリリース以降の質問には正しく回答することができなかった。

In [ ]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 8192

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
with open("/content/lecture-ai-engineering/day3/homework/data/weather.txt", "r") as f:
  raw_writedown = f.read()

In [ ]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[0])

ドキュメントサイズ:  3
ドキュメントの例: 
 2025年5月7日（水）の東京（江戸川区）の天気は、概ね晴れで、最高気温は25°C、最低気温は15°Cでした


In [ ]:
# Retrievalの実行
question = "2025年5月7日の東京の天気は大雨ですか?"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[87.10651397705078, 70.21236419677734, 56.956459045410156]]


In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 87.10651397705078)
2025年5月7日（水）の東京（江戸川区）の天気は、概ね晴れで、最高気温は25°C、最低気温は15°Cでした 


取得したドキュメント2: (Score: 70.21236419677734)
ただし、午後6時まで強風注意報と波浪注意報が発表されており、風が強い一日となりました 


取得したドキュメント3: (Score: 56.956459045410156)
 




In [ ]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] 2025年5月7日の東京の天気は大雨ですか?"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(response)

日本語で回答します。

2025年5月7日の東京の天気は、概ね晴れでした。強風注意報と波浪注意報が発表されていたが、大雨は報告されていません。


### 結果(初期RAG実装)
2025年5月7日の東京の天気の情報が記載されたファイルを受け取ることで正しい回答をすることができた。

# 質問②
映画「ブレードランナー」の原作は？

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "映画「ブレードランナー」の原作は？"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(response)

映画「ブレードランナー」の原作は、フィリップ・K・ディックの小説「ドゥーン」(Do Androids Dream of Electric Sheep?)です。


## 結果(ベースモデル)
概ね正しい回答だったが、一部誤った情報が混入していた。原題は正しいが、日本語名の「ドゥーン」というタイトルは誤り。おそらくフランク・ハーバートのSF小説『デューン』(原題:Dune)と混同されている可能性がある。

In [ ]:
with open("/content/lecture-ai-engineering/day3/homework/data/blade_runner.txt", "r") as f:
  raw_writedown = f.read()

In [ ]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[10])

ドキュメントサイズ:  394
ドキュメントの例: 
 地球に残った人々は酸性雨の降りしきる、高層ビル群が立ち並んだ人口過密の大都市での生活を強いられていた


In [ ]:
# Retrievalの実行
question = "映画「ブレードランナー」の原作は？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[84.97337341308594, 73.03462982177734, 78.98394012451172, 73.3218765258789, 65.69698333740234, 71.43521118164062, 68.12216186523438, 68.43009185791016, 78.60972595214844, 65.94720458984375, 68.44554901123047, 68.82671356201172, 77.031494140625, 76.5520248413086, 70.7385482788086, 70.17691040039062, 66.17940521240234, 67.02171325683594, 70.58080291748047, 66.0847396850586, 69.47716522216797, 66.08058166503906, 72.00646209716797, 69.98555755615234, 69.17044067382812, 68.93862915039062, 67.42508697509766, 68.16934967041016, 68.09148406982422, 68.40093994140625, 67.40435028076172, 73.99481201171875, 78.7802734375, 72.60564422607422, 67.4753646850586, 62.67689514160156, 69.73148345947266, 70.6549301147461, 68.72547912597656, 66.76599884033203, 65.59300231933594, 69.89159393310547, 78.65481567382812, 69.4727554321289, 71.38143157958984, 67.3603515625, 63.33222198486328, 70.7595443725586, 66.60405731201172, 67.20729064941406, 71.09869384765625, 70.28739929199219, 63.37763595581055, 69.466033

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 84.97337341308594)
『ブレードランナー』（原題：Blade Runner）は、1982年のアメリカ合衆国のSF映画 


取得したドキュメント2: (Score: 84.89627838134766)
この小説を元にウィリアム・S・バロウズは映画化用の翻案として『Blade Runner (a movie)[注釈 4]』（1979年、訳題『映画:ブレードランナー』）を執筆した 


取得したドキュメント3: (Score: 83.92424774169922)
「ブレードランナー」という名称は、SF作家アラン・E・ナースの小説『The Bladerunner』（1974年）において「非合法医療器具（blade）の運び屋（runner）」という意味で登場する 


取得したドキュメント4: (Score: 83.56727600097656)
「ブレードランナー」と「レプリカント」
本作に登場する「ブレードランナー」と「レプリカント（英語版）」は、原作には登場しない映画オリジナル用語である 


取得したドキュメント5: (Score: 82.80364227294922)
と同時に、スコットが本来意図した『ブレードランナー』を見たいという要望が高まり、ワーナーは彼に再び本作の編集を依頼[4] 




In [ ]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] 映画「ブレードランナー」の原作は？"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(response)

映画「ブレードランナー」の原作は、フィリップ・K・ディックの小説「Do Androids Dream of Electric Sheep?」（1968年）です。


## 結果(初期RAG実装)
Wikipediaの情報を受け取ることで正しい回答をすることができた。

# 質問③
映画「ブレードランナー」に出てくる架空の飛行車の総称は？

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "映画「ブレードランナー」に出てくる架空の飛行車の総称は？"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(response)

映画「ブレードランナー」に出てくる架空の飛行車の総称は「spinner」です。


## 結果(ベースモデル)
ベースモデルの段階で正しく答えることができた。

In [ ]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] 映画「ブレードランナー」に出てくる架空の飛行車の総称は？"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(response)

映画「ブレードランナー」に出てくる架空の飛行車の総称は「spinner」という名称です。


## 結果(初期RAG実装)
RAGを行った場合も間違えることなく正しい答えを返した。

# 質問④
映画『ブレードランナー』に登場する、日本語で「入り陽傾く」という音声とともに芸者が現れる広告について、次の問いに答えなさい。この広告は誰のアイデアとされ、また劇中ではどのような製品として描かれているか？実際の製品との違いにも言及しなさい。

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "映画『ブレードランナー』に登場する、日本語で「入り陽傾く」という音声とともに芸者が現れる広告について、次の問いに答えなさい。この広告は誰のアイデアとされ、また劇中ではどのような製品として描かれているか？実際の製品との違いにも言及しなさい。"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(response)

映画『ブレードランナー』に登場する広告は、ジョン・ハンソン（John Harrison）がデザインしたものとされます。ハンソンは、映画のプロデューサーであるハンター・スチュワート（Harrison Starr）と共にこの広告をデザインしたと考えられます。

この広告は、劇中では「Mitsubishi WaYo」という製品として描かれています。WaYoは、未来の自動車であり、芸者が乗車することで、乗客に性的サービスを提供するという設定です。

実際の製品との違いとして、映画ではWaYoが女性の芸者を乗車させるという設定ですが、実際の自動車では女性の芸者が乗車することはないと考えられます。また、映画ではWaYoが未来の技術を活用した高級自動車として描かれていますが、実際の自動車ではそのような技術を活用した製品は存在しません。


In [ ]:
# @title 評価実装
gold_answer = "日本語でイリヒカタムク（入り陽傾く）という声と芸者が現れる「強力わかもと」の広告はシド・ミードのアイデアとされ、実際の製品は胃腸薬だが避妊薬という設定で登場する"

from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

def openai_generator(query):

        messages = [
                    {
                        "role": "user",
                        "content": query
                    }
                ]

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content

def evaluate_answer_accuracy(query, response, reference):

    template_accuracy1 = (
          "Instruction: You are a world class state of the art assistant for rating "
          "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
          "Say 4, if User Answer is full contained and equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
          "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
          "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
          "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
          "### Question: {query}\n"
          "### {answer0}: {sentence_inference}\n"
          "### {answer1}: {sentence_true}\n"
          "The rating is:\n"
      )
    template_accuracy2 = (
          "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
          "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
          "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
          "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
          "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
          "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
          "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
          "Question: {query}\n\n"
          "{answer0}: {sentence_inference}\n\n"
          "{answer1}: {sentence_true}\n\n"
          "Rating: "
      )

    score1 = openai_generator(
                template_accuracy1.format(
                      query=query,
                      answer0="User Answer",
                      answer1="Reference Answer",
                      sentence_inference=response,
                      sentence_true=reference,
                    )
                )
    try:
      score1 = int(score1)
    except:
      print("Failed")
      score1 = 0

    score2 = openai_generator(
                template_accuracy2.format(
                        query=query,
                        answer0="Reference Answer",
                        answer1="User Answer",
                        sentence_inference=reference,
                        sentence_true=response,
                    )
                  )

    try:
      score2 = int(score2)
    except:
      print("Failed")
      score2 = 0


    return (score1 + score2) / 2

In [ ]:
# 評価
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

0.0


## 結果(ベースモデル)
全くデタラメな答えになった。gpt-4oによる評価も0.0になった。

In [ ]:
# Retrievalの実行
question = "映画『ブレードランナー』に登場する、日本語で「入り陽傾く」という音声とともに芸者が現れる広告について、次の問いに答えなさい。この広告は誰のアイデアとされ、また劇中ではどのような製品として描かれているか？実際の製品との違いにも言及しなさい。"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[77.95146942138672, 69.29281616210938, 71.28653717041016, 69.84960174560547, 63.96390151977539, 67.35643768310547, 68.87885284423828, 64.78221130371094, 73.05975341796875, 65.55174255371094, 66.98918151855469, 64.72613525390625, 72.12278747558594, 73.544189453125, 67.40434265136719, 68.34663391113281, 67.8724136352539, 66.80297088623047, 66.95085144042969, 64.68270111083984, 67.3154067993164, 65.44209289550781, 70.44393920898438, 68.55610656738281, 67.19405364990234, 65.88933563232422, 67.24413299560547, 67.40733337402344, 68.39048767089844, 67.08695983886719, 65.33653259277344, 71.86626434326172, 73.6134033203125, 69.92147827148438, 66.24465942382812, 65.22196960449219, 67.43730926513672, 67.62813568115234, 68.19920349121094, 67.71781158447266, 66.30183410644531, 67.80362701416016, 73.96691131591797, 67.93550872802734, 69.57263946533203, 67.15159606933594, 65.02196502685547, 70.53275299072266, 68.81698608398438, 67.12095642089844, 69.33956146240234, 68.8548583984375, 65.4726943969726

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 79.27154541015625)
日本語でイリヒカタムク（入り陽傾く）という声と芸者が現れる「強力わかもと」の広告はシド・ミードのアイデアとされ[15]、実際の製品は胃腸薬だが避妊薬という設定で登場する[32][33][注釈 14] 


取得したドキュメント2: (Score: 77.95146942138672)
『ブレードランナー』（原題：Blade Runner）は、1982年のアメリカ合衆国のSF映画 


取得したドキュメント3: (Score: 77.73279571533203)
「ブレードランナー」という名称は、SF作家アラン・E・ナースの小説『The Bladerunner』（1974年）において「非合法医療器具（blade）の運び屋（runner）」という意味で登場する 


取得したドキュメント4: (Score: 76.79956817626953)
→以上の詳細については「ブレードランナー (アルバム)」を参照
日本語吹替
役名	俳優	日本語吹替
TBS版	ザ・シネマ版
リック・デッカード	ハリソン・フォード	堀勝之祐	磯部勉
ロイ・バッティ	ルトガー・ハウアー	寺田農	谷口節
レイチェル	ショーン・ヤング	戸田恵子	岡寛恵
ガフ	エドワード・ジェームズ・オルモス	池田勝	佳月大人
H・ブライアント	M・エメット・ウォルシュ	神山卓三	浦山迅
プリス	ダリル・ハンナ	高島雅羅	小島幸子
J・F・セバスチャン	ウィリアム・サンダーソン	村越伊知郎	村治学
リオン	ブライオン・ジェームズ	大宮悌二	中村浩太郎
エルドン・タイレル	ジョー・ターケル	大木民夫	小島敏彦
ゾーラ	ジョアンナ・キャシディ	横尾まり	森夏姫
ハンニバル・チュウ	ジェームズ・ホン	千葉順二	浦山迅
ホールデン	モーガン・ポール（英語版）	二瓶秀雄	高岡瓶々
スシバーの店主	ロバート・オカザキ	千葉順二	小島敏彦
その他		竹口安芸子	中司ゆう花
小橋知子
佐藤拓也
片貝薫
演出		河村常平	伊達康将
翻訳		岩本令	岸田恵子
録音・調整		切金潤	オムニバス・ジャパン
効果		遠藤堯雄
桜井俊哉	
プロデューサー		上田正人
（TBS）	井伊直子
飯森盛良
制作		東北新社
TBS	ザ・シネマ
東北新社
初回放送		1986

In [ ]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] 映画『ブレードランナー』に登場する、日本語で「入り陽傾く」という音声とともに芸者が現れる広告について、次の問いに答えなさい。この広告は誰のアイデアとされ、また劇中ではどのような製品として描かれているか？実際の製品との違いにも言及しなさい。"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(response)

日本語で「入り陽傾く」という音声とともに芸者が現れる広告は、シド・ミードのアイデアとされます。実際の製品は胃腸薬ですが、劇中では避妊薬として描かれています。


In [ ]:
# 評価 (openai apikeyがある場合のみ実行)
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

4.0


## 結果 (初期RAG実装)
ドキュメントを渡したことで、複雑な問題に対しても完璧な答えを返してきた。gptによる評価も高いスコアをつけた。

# 質問⑤
映画『ブレードランナー』における主人公リック・デッカードの正体について、作中では「レプリカントではないか」という解釈が存在します。これについて次の設問に答えなさい。

設問：
デッカード＝レプリカント説の根拠と、それに対する反論の両方を挙げたうえで、この説に関する最終的な評価を述べなさい。

In [ ]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=1024,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question = """
映画『ブレードランナー』における主人公リック・デッカードの正体について、
作中では「レプリカントではないか」という解釈が存在します。
これについて次の設問に答えなさい。

設問：
デッカード＝レプリカント説の根拠と、それに対する反論の両方を挙げたうえで、
この説に関する最終的な評価を述べなさい。
"""
response = generate_output(question, system_prompt)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(response)

映画『ブレードランナー』におけるデッカードの正体について、作中では「レプリカントではないか」という解釈が存在します。この説の根拠と反論を挙げて、最終的な評価を述べます。

根拠：

1. デッカードの記憶喪失：デッカードは、映画の冒頭で記憶喪失状態に陥っており、過去の経験や自分の身元を忘れていた。この点で、レプリカント説の支持者たちは、デッカードがレプリカントである可能性があると考える。
2. デッカードの能力：デッカードは、レプリカントのように、非常に高い身体能力と、瞬間的な判断力を示した。また、彼は、レプリカントの能力に似た、瞬間的な記憶喪失や、自己の存在を疑うことがある。
3. ヴォイテックの言動：ヴォイテックは、デッカードに対して、レプリカントであると疑問視する言動を示した。この点で、レプリカント説の支持者たちは、ヴォイテックがデッカードのレプリカントであることを知っていたと考える。

反論：

1. デッカードの感情：デッカードは、映画の冒頭で、強い感情を示し、レプリカントのように、感情を制御できなかった。しかし、レプリカントは、感情を制御することができず、デッカードの感情は、レプリカントのように、感情を制御できたと考えることができる。
2. デッカードの自我：デッカードは、映画の冒頭で、自我を疑うことがあり、レプリカントのように、自我を失うことがある。しかし、デッカードは、自我を回復し、レプリカントのように、自我を失うことができなかったと考えることができる。
3. ヴォイテックの言動の意味：ヴォイテックの言動は、デッカードに対して、レプリカントであると疑問視する言動であったが、ヴォイテックは、デッカードがレプリカントであることを知っていたと考えることができる。しかし、ヴォイテックの言動は、デッカードに対して、レプリカントであると疑問視する言動であり、デッカードがレプリカントであることを示すものではなかったと考えることができる。

最終的な評価：

デッカード＝レプリカント説は、映画『ブレードランナー』におけるデッカードの正体について、レプリカントである可能性があると考える説である。しかし、この説には、反論も存在する。デッカードの感情、自我、ヴォイテックの言動など、映画の要素を考慮すると、デッカードがレプリカントであると考えるには、十分な根拠はないと考える

In [ ]:
gold_answer = """
デッカードがレプリカントであるという説には、次のような根拠がある。
まず『ディレクターズ・カット』や『ファイナル・カット』において挿入されたユニコーンの夢のシーンは、ラストの折り紙と連動し、ガフがデッカードの内面（夢の内容）を知っていることを示唆している。このことから、デッカードの記憶は人工的に植え付けられたものである＝レプリカントである、という推論が導かれる。また、監督のリドリー・スコット自身が「デッカードはレプリカントだ」と明言しており、これが公式見解とされる場合もある。さらに、デッカードの赤目や写真の演出も、レプリカントであることをほのめかすものである。

一方で、脚本家ハンプトン・ファンチャーは人間であることを想定して物語を書いており、俳優ハリソン・フォードも当初は人間として演じていた。観客の共感を呼ぶためには人間として描いた方が自然であり、物語構造上もレプリカントであると断定しない方が意義深いという意見もある。

これらを総合すると、『ブレードランナー』はあえて主人公の正体を明示せず、観客に人間性や存在の本質を問いかける構造となっている。したがって、最終的には観客の解釈に委ねられており、「デッカード＝レプリカント説」は強く示唆されつつも断定はできないというのが妥当な評価である。
"""

In [ ]:
# 評価 (openai apikeyがある場合のみ実行)
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

2.0


## 結果(ベースモデル)
複雑な問題に対して、根拠を示した考察の回答をすることができた。ただ、模範回答と比較すると決定的ではない為、gptによる評価も2.0にとどまった。

In [ ]:
# Retrievalの実行
question = """
映画『ブレードランナー』における主人公リック・デッカードの正体について、
作中では「レプリカントではないか」という解釈が存在します。
これについて次の設問に答えなさい。

設問：
デッカード＝レプリカント説の根拠と、それに対する反論の両方を挙げたうえで、
この説に関する最終的な評価を述べなさい。
"""

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[77.48275756835938, 72.27427673339844, 75.47857666015625, 76.84175109863281, 69.4695053100586, 76.20597076416016, 72.48274993896484, 74.5888442993164, 80.609130859375, 68.22396850585938, 68.0257568359375, 73.8626937866211, 79.60914611816406, 80.34803009033203, 77.71992492675781, 79.58567810058594, 73.85002136230469, 76.40558624267578, 73.30496215820312, 77.02179718017578, 75.70097351074219, 70.82400512695312, 77.9482421875, 71.57840728759766, 71.89329528808594, 74.69166564941406, 70.4400634765625, 75.33910369873047, 76.5419921875, 76.99623107910156, 76.07328796386719, 80.54889678955078, 76.45341491699219, 78.89627075195312, 74.56866455078125, 64.98296356201172, 72.10601806640625, 77.14765167236328, 70.07897186279297, 69.27262878417969, 67.99108123779297, 70.54533386230469, 81.79338836669922, 77.91639709472656, 74.8691177368164, 70.39608001708984, 65.51922607421875, 70.86781311035156, 71.46482849121094, 68.77013397216797, 75.25623321533203, 71.69305419921875, 65.0787124633789, 70.58951

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 85.44410705566406)
以上のように諸々の経緯はあるが、「デッカード＝レプリカント説」を断定出来るような描写はどの版にも存在しない 


取得したドキュメント2: (Score: 83.76325988769531)
スコット自身は、2000年にイギリスのChannel 4 Televisionが制作したドキュメンタリー『ON THE EDGE OF BLADE RUNNER』のインタビューにおいて「デッカードはレプリカントだ」と明言している 


取得したドキュメント3: (Score: 83.39350891113281)
しかし、後にフォードは私は、ずっと彼がレプリカントだと知っていましたとも証言しており、レプリカントは自分自身を人間だと信じたいものであり、デッカードを演じて抗いたい思いもあったと述べている[25] 


取得したドキュメント4: (Score: 82.70329284667969)
「ブレードランナー」と「レプリカント」
本作に登場する「ブレードランナー」と「レプリカント（英語版）」は、原作には登場しない映画オリジナル用語である 


取得したドキュメント5: (Score: 81.98631286621094)
なお、ブライアントがデッカードにレプリカントのプロフィールを説明しているシーンは、スタジオセット内で撮影されている[71] 




In [ ]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"""
[参考資料]\n{references}\n\n
[質問] 映画『ブレードランナー』における主人公リック・デッカードの正体について、
作中では「レプリカントではないか」という解釈が存在します。
これについて次の設問に答えなさい。

設問：
デッカード＝レプリカント説の根拠と、それに対する反論の両方を挙げたうえで、
この説に関する最終的な評価を述べなさい。
"""
response = generate_output(question, system_prompt)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(response)

デッカード＝レプリカント説は、映画『ブレードランナー』における主人公リック・デッカードの正体について、レプリカントであるという解釈を指します。

根拠として、以下のような点が挙げられます。

* スコット自身が、2000年のドキュメンタリー『ON THE EDGE OF BLADE RUNNER』で「デッカードはレプリカントだ」と明言している。
* フォードが、デッカードを演じた際に、レプリカントであることを知っていたと証言している。

反論として、以下のような点が挙げられます。

* 作中では、ブライアントがデッカードにレプリカントのプロフィールを説明しているシーンは、スタジオセット内で撮影されているため、実際の設定ではレプリカントであると示唆する描写はない。
* デッカードの人物像や行動も、レプリカントであると解釈するには、十分な根拠が存在しない。

最終的な評価として、デッカード＝レプリカント説は、作中での描写や設定に基づいて、明確に示唆される解釈ではないと考えられます。スコットやフォードの発言や、作中での描写に基づいて、デッカードの正体についての解釈は、複数の可能性を持つと考えられます。


In [ ]:
# 評価 (openai apikeyがある場合のみ実行)
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

2.0


## 結果 (初期RAG実装)
ドキュメントを渡したことで、メタ的な観点が取り入れられ模範解答に近づいた印象があった。gptの評価としては変わらなかった。

### 文脈を考慮したチャンク化の導入

In [ ]:
# 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"""
[参考資料]\n{references}\n\n
[質問] 映画『ブレードランナー』における主人公リック・デッカードの正体について、
作中では「レプリカントではないか」という解釈が存在します。
これについて次の設問に答えなさい。

設問：
デッカード＝レプリカント説の根拠と、それに対する反論の両方を挙げたうえで、
この説に関する最終的な評価を述べなさい。
"""
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(response)

映画『ブレードランナー』における主人公リック・デッカードの正体について、作中では「レプリカントではないか」という解釈が存在します。

デッカード＝レプリカント説の根拠として、以下のような点が挙げられます。

* スコット監督が2000年に行ったインタビューにおいて、「デッカードはレプリカントだ」と明言している。
* ユニコーンの夢シーンにおいて、デッカードがレプリカントであるという可能性を示唆している。
* デッカードがレプリカントであるというアイデアは撮影途中でスコットが思いついたことで、当初はそのように考えて撮影されていなかったとされ、フォードもレプリカントだとは指示を受けておらず、撮影時には人間として演じたという。

一方、デッカード＝レプリカント説に対する反論として、以下のような点が挙げられます。

* フォードが、デッカードを演じた際には人間として演じたと述べている。
* ファンチャー脚本家が、デッカードを人間と考えることを唱えている。
* フォードが、デッカードをレプリカントであると知っていたと述べているが、レプリカントは自分自身を人間だと信じたいものであり、デッカードを演じて抗いたい思いもあったと述べている。

以上のように、デッカード＝レプリカント説には根拠と反論の両方があります。ただし、最終的な評価として、デッカードがレプリカントであるという説は、スコット監督の発言やユニコーンの夢シーンなど、映画中の暗示や暗黙の内容に基づいて、より可能性を示唆するものと言えます。ただし、映画の作者や関係者らの意見は異なるため、最終的な評価は個々の解釈に委ねられます。


In [ ]:
# 評価 (openai apikeyがある場合のみ実行)
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

3.0


## 結果 (文脈付きチャンク化によるRAG)

文脈を含むチャンク化により、モデルの回答の方向性に明確な改善が見られた。メタ的な視点に加え、最終的な評価もより具体的になった。gptの評価も向上した。




In [ ]:
%cd lecture-ai-engineering

/content/lecture-ai-engineering


In [ ]:
!git config --global user.name "yuki-mtmr"
!git config --global user.email "yuki.matsumori.exam@gmail.com"

In [ ]:
!git add .
!git commit -m "add:第三回宿題"
!git push origin master

[master 35c9398] add:第三回宿題
 3 files changed, 12964 insertions(+)
 create mode 100644 day3/homework/data/blade_runner.txt
 create mode 100644 day3/homework/data/weather.txt
 create mode 100644 "day3/homework/\347\254\254\344\270\211\345\233\236\345\256\277\351\241\214.ipynb"
fatal: could not read Username for 'https://github.com': No such device or address
